# Exemplo

Suponhamos que estamos desenvolvendo um cliente que fará uma série de requests a uma API. Esperamos resultados diferentes para cada request, mas cada todas elas têm exigem algum housekeeping: error handling, logging, etc. O Template Method pode ser uma maneira de evitar a repeticao em situacoes como essa.

In [2]:
import abc
import requests
from copy import deepcopy

class MockResponse:
    '''
    Creates mock responses to http requests
    '''
    def __init__(self, status: int, data: dict):
        self.status = status
        self.data = data
        
    def json(self) -> dict:
        return self.data
    
    @staticmethod
    def mock_post_request(url: str, data: dict) -> 'MockResponse':
        '''
        Mocks a post request
        '''
        if url == 'mockserver.co.uk/login':
            if data.get("password") == "1234":
                return MockResponse(200, {"message": "Logged in", "account_id": "1111"})
            else:
                return MockResponse(401, {"message": "Incorrect username or email"})
        if url == 'mockserver.co.uk/addstuff':
            return MockResponse(200, {"message":"Added stuff", "stuff": data})
    
    @staticmethod   
    def mock_get_request(url: str) -> 'MockResponse':
        '''
        Mocks a get request
        '''
        if url == 'mockserver.co.uk/indexstuff':
            return MockResponse(200, {"data": ['some','random','stuff']})
        if url == 'mockserver.co.uk/somethingbad':
            raise requests.ConnectionError('Something bad happened')
        else:
            return MockResponse(404, {"message": "Not found"})
        

'''
A implementacao do Template Method comeca aqui
'''

class APIRequest(abc.ABC):
    
    def __init__(self, url: str, method: str, data: dict={}):
        self.url = url
        self.method = method
        self.data = data
        
    def get_data_without_password(self):
        data = deepcopy(self.data)
        data.pop('password', None)
        return data
    
    def log_info(self):
        print(f"LOG LEVEL: INFO - URL: {self.url} METHOD: {self.method} DATA: {self.get_data_without_password()}")
    
    def log_error(self, message: str):
        print(f"LOG LEVEL: ERROR - URL: {self.url} METHOD: {self.method} DATA: {self.get_data_without_password()} MESSAGE: {message}")
        
    def on_internal_error(self):
        '''
        Hook
        '''
        pass
    
    def on_http_error(self, response: dict):
        '''
        Hook
        '''
        pass
    
    def validate(self, data: dict={}) -> bool:
        '''
        validates the payload before making the request
        '''
        return True
    
    def on_validation_failure(self):
        '''
        Hook
        '''
        pass
    
    def on_validation_success(self):
        '''
        Hook
        '''
        pass
    
    def send_request(self) -> dict:
        if self.method == 'post':
            return MockResponse.mock_post_request(self.url, self.data)
        return MockResponse.mock_get_request(self.url)
    
    @abc.abstractmethod
    def handle_success(self, data: dict):
        pass
    
    def make_request(self):
        '''
        The template method
        '''
        try:
            is_valid = self.validate()
            if not is_valid:
                self.log_error('Validation failed')
                self.on_validation_failure()
                return
            self.on_validation_success()
            response = self.send_request()
            if response.status != 200:
                self.on_http_error(response)
                error = response.json()
                self.log_error(error.get("message"))
                return
            self.log_info()
            self.handle_success(response.json())
        except:
            self.log_error("Internal error")
            self.on_internal_error()

'''
Exemplos de uso
'''

'''
Caso básico:
'''

class SomeGetRequest(APIRequest):
    
    def __init__(self):
        super().__init__('mockserver.co.uk/indexstuff', 'get')
        
    def handle_success(self, response: dict):
        print("Successfully retrieved data")
        for row in response["data"]:
            print(row)

            
print('Exemplo de get request')
s1 = SomeGetRequest()
s1.make_request()

'''
Internal error:
'''

class BadGetRequest(APIRequest):
    
    def __init__(self):
        super().__init__('mockserver.co.uk/somethingbad', 'get')
        
    def handle_success(self, response: dict):
        print("We won't need this")

print()
print('Exemplo de internal error')
b1 = BadGetRequest()
b1.make_request()

'''
Com Hooks e validation:
'''

            
class LoginRequest(APIRequest):
    def __init__(self, payload: dict={}):
        super().__init__('mockserver.co.uk/login', 'post', payload)
        
    def validate(self):
        return bool(self.data.get("password")) and bool(self.data.get("username"))
    
    def on_validation_failure(self):
        print('Please enter a password')
    
    def on_validation_success(self):
        print('Attempting to log in...')
        
    def on_http_error(self, response: dict):
        if response.status == 401:
            print('Incorrect email or passwod')
        else: 
            print('Unable to log in')
        
    def handle_success(self, data):
        print(f"Your account ID is {data.get('account_id')}")


print()
print('Exemplos com hooks e validation')
print('Request without password:')
r1 = LoginRequest({"username": "test", "password": ""})
r1.make_request()
print()
print('Request with a valid password:')
r2 = LoginRequest({"username": "test", "password": "1234"})
r2.make_request()
print()
print('Request with an invalid password:')
r3 = LoginRequest({"username": "test", "password": "123"})
r3.make_request()
    

    

    
    
        
    



Exemplo de get request
LOG LEVEL: INFO - URL: mockserver.co.uk/indexstuff METHOD: get DATA: {}
Successfully retrieved data
some
random
stuff

Exemplo de internal error
LOG LEVEL: ERROR - URL: mockserver.co.uk/somethingbad METHOD: get DATA: {} MESSAGE: Internal error

Exemplos com hooks e validation
Request without password:
LOG LEVEL: ERROR - URL: mockserver.co.uk/login METHOD: post DATA: {'username': 'test'} MESSAGE: Validation failed
Please enter a password

Request with a valid password:
Attempting to log in...
LOG LEVEL: INFO - URL: mockserver.co.uk/login METHOD: post DATA: {'username': 'test'}
Your account ID is 1111

Request with an invalid password:
Attempting to log in...
Incorrect email or passwod
LOG LEVEL: ERROR - URL: mockserver.co.uk/login METHOD: post DATA: {'username': 'test'} MESSAGE: Incorrect username or email
